In [1]:
from __future__ import annotations

import json
import cProfile
import pstats

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [6]:
with cProfile.Profile() as pr:
  rec = await data_source.load(data, cache_with_db=True)
  
  stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│ ❱ 2   rec = await data_source.load(data, cache_with_db=True)                                     │
│   3                                                                                              │
│   4   stats = pstats.Stats(pr)                                                                   │
│   5                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:580 in load                   │
│                                                                                                  │
│   577 │   │   │   db if cache_with_db and db is not None else DB() if cache_with_db else None    │
│   578 │   │   )                                                                                  │
│   579 │   │                                                                                      │
│ ❱ 580 │   │   rec = await _load_record(self.rec, self, data, self._obj_cache, db_cache)          │
│   581 │   │                                                                                      │
│   582 │   │   if db_cache is not None:                                                           │
│   583 │   │   │   db_cache[type(rec)] |= rec                                                     │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:474 in _load_record           │
│                                                                                                  │
│   471 │   }                                                                                      │
│   472 │                                                                                          │
│   473 │   rec_dict: dict[Col | RelSet, Any] = {a[0]: a[2] for a in attrs.values()}               │
│ ❱ 474 │   rec = rec_type._from_partial_dict(rec_dict)                                            │
│   475 │                                                                                          │
│   476 │   if rec._index in py_cache[rec_type]:                                                   │
│   477 │   │   return py_cache[rec_type][rec._index]                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:1428 in _from_partial_dict           │
│                                                                                                  │
│   1425 │   │   │   p.name: data.get(p if not name_keys else p.name, State.undef)  # type: ignor  │
│   1426 │   │   │   for p in cls._props.values()                                                  │
│   1427 │   │   }                                                                                 │
│ ❱ 1428 │   │   return cls(**args, _loader=loader)                                                │
│   1429 │                                                                                         │
│   1430 │   def __repr__(self) -> str:  # noqa: D105                                              │
│   1431 │   │   return self._to_dict(name_keys=True, only_loaded=False).__repr__()                │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py

In [ ]:
rec

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")